In [ ]:
import os
from pathlib import Path
from PIL import Image

def find_bounding_box(mask_image):
    """
    Находит ограничивающий прямоугольник (bounding box) для ненулевых пикселей в маске.
    
    Возвращает 4-кортеж (left, upper, right, lower) для использования в Image.crop().
    """
    # Преобразуем маску в черно-белый режим '1' (1-битный пиксель, 0 или 1) для Pillow.
    # Это гарантирует, что getbbox() найдет границы объекта (ненулевые пиксели).
    if mask_image.mode != '1':
        # Конвертируем в L (градации серого), затем в '1' с порогом (например, 128)
        mask_image_bw = mask_image.convert('L').point(lambda x: 0 if x < 128 else 255, mode='1')
    else:
        mask_image_bw = mask_image

    # Получаем bounding box. getbbox() возвращает кортеж (left, upper, right, lower),
    # который охватывает все ненулевые пиксели.
    bbox = mask_image_bw.getbbox()
    
    return bbox, mask_image_bw # Возвращаем bbox и преобразованную маску для дальнейшего использования

def crop_and_mask_image(dataset_root_dir="dataset", output_root_dir="masked_png_dataset"):
    """
    Проходит по папкам, находит изображения и маски, создает PNG с прозрачностью 
    по маске, обрезает по bounding box и сохраняет результат.
    """
    
    # Создаем корневую директорию для обрезанных изображений, если ее нет
    Path(output_root_dir).mkdir(exist_ok=True)
    
    # Используем pathlib для удобного обхода папок
    dataset_path = Path(dataset_root_dir)
    
    # Итерируем по всем подпапкам первого уровня (папки инструментов)
    for tool_folder in dataset_path.iterdir():
        if tool_folder.is_dir():
            print(f"Обработка папки: {tool_folder.name}")
            if tool_folder.name in ["Инструменты с линейкой", "Групповые для тренировки"]:
                print(f"Пропускаем папку: {tool_folder.name}")
                continue
            
            # Создаем соответствующую выходную папку
            output_tool_path = Path(output_root_dir) / tool_folder.name
            output_tool_path.mkdir(exist_ok=True)
            
            # Путь к папке с масками
            mask_dir = tool_folder / "razmetka_bbox_mask"
            
            if not mask_dir.is_dir():
                print(f"   Пропущена: папка масок {mask_dir} не найдена.")
                continue

            # Итерируем по всем JPG-изображениям в папке инструмента
            for image_path in tool_folder.glob("*.JPG"):
                try:
                    # Имя файла изображения без расширения
                    image_stem = image_path.stem 
                    
                    # Ищем соответствующий файл маски
                    mask_filename = f"{image_stem}_mask_00.png"
                    mask_path = mask_dir / mask_filename
                    
                    if not mask_path.exists():
                        print(f"   Пропущено: маска {mask_filename} не найдена для {image_path.name}")
                        continue
                        
                    # 1. Загрузка изображения и маски
                    img = Image.open(image_path).convert("RGB") # Исходное изображение
                    mask_orig = Image.open(mask_path)           # Маска
                    
                    # Убедимся, что размеры изображения и маски совпадают
                    if img.size != mask_orig.size:
                        print(f"   Ошибка: Размеры {image_path.name} ({img.size}) и {mask_path.name} ({mask_orig.size}) не совпадают. Пропущено.")
                        continue
                        
                    # 2. Поиск ограничивающего прямоугольника
                    bbox, mask_bw = find_bounding_box(mask_orig)
                    
                    if bbox is None:
                        print(f"   Предупреждение: Bounding box не найден (маска пустая) для {image_path.name}. Пропущено.")
                        continue
                        
                    # --- НОВЫЕ ШАГИ ДЛЯ СОЗДАНИЯ ПРОЗРАЧНОГО PNG ---
                    
                    # 3. Создание прозрачного изображения
                    # a. Конвертируем изображение в режим RGBA (RGB + Alpha)
                    img_rgba = img.convert("RGBA") 
                    
                    # b. Изменяем режим маски на 'L' (градации серого), чтобы использовать ее как альфа-канал
                    # Мы используем mask_bw, преобразованную в find_bounding_box, но преобразуем ее обратно в 'L'
                    # для корректного использования в качестве альфа-канала.
                    # '1' режим от find_bounding_box уже гарантирует, что ненулевые пиксели будут объектом.
                    # Просто конвертируем в 'L' (градации серого), где 0 - прозрачно, 255 - непрозрачно.
                    # Поскольку маска была обработана в find_bounding_box, используем ее.
                    alpha_mask = mask_bw.convert('L')
                    
                    # c. Применяем маску как альфа-канал
                    # split() разделяет RGBA на R, G, B, A каналы. 
                    # Мы заменяем оригинальный альфа-канал (A) на нашу маску (alpha_mask)
                    img_rgba.putalpha(alpha_mask) 
                    
                    # d. Обрезка изображения и маски
                    # Обрезаем прозрачное RGBA изображение по найденному bounding box.
                    cropped_img_png = img_rgba.crop(bbox)
                    
                    # --- КОНЕЦ НОВЫХ ШАГОВ ---
                    
                    # 4. Сохранение обрезанного изображения в PNG
                    output_filename = f"{image_stem}_masked.png" # Имя файла с расширением PNG
                    output_path = output_tool_path / output_filename
                    
                    # Уменьшаем размер изображения до 512 по наибольшей стороне (если нужно)
                    cropped_img_png.thumbnail((512, 512), Image.Resampling.LANCZOS)
                    # Сохраняем в формате PNG. Pillow автоматически сохранит его с прозрачностью.
                    cropped_img_png.save(output_path, "PNG") 
                    
                    print(f"   Успешно обработано: {image_path.name} -> {output_filename} (PNG), BBox: {bbox}")

                except Exception as e:
                    print(f"   Произошла ошибка при обработке {image_path.name}: {e}")

    print("Обработка завершена! 🎉")

# Запуск функции
if __name__ == "__main__":
    # Укажите корневую папку с вашими данными и папку для сохранения результатов
    crop_and_mask_image(
        dataset_root_dir="../dataset/dataset", 
        output_root_dir="../dataset/masked_png_dataset" # Изменено имя папки для лучшего соответствия результату
    )

In [ ]:
import os
import random
from pathlib import Path
from PIL import Image

# 1. Параметры
CANVAS_SIZE = (768, 768)
TARGET_TOOL_FOLDERS = ["1 Отвертка «-»", "2 Отвертка «+»", "3 Отвертка на смещенный крест"]
MAX_RANDOM_TOOLS = 5
# Корень исходного датасета для фоновых изображений
RAW_DATASET_ROOT = "../dataset/dataset" 

# --- Вспомогательные функции для фонов ---

def get_background_image_paths(raw_root_dir, target_folders):
    """Собирает пути ко всем JPG-изображениям, исключая те, что в целевых папках."""
    all_background_paths = []
    
    for tool_folder in Path(raw_root_dir).iterdir():
        if tool_folder.is_dir() and tool_folder.name not in target_folders:
            # Собираем все JPG-файлы в этой папке
            all_background_paths.extend(list(tool_folder.glob("*.JPG")))
            
    return all_background_paths

def generate_random_background(background_paths, canvas_size):
    """
    Выбирает случайное фоновое изображение и обрабатывает его до размера холста (1024x1024).
    """
    if not background_paths:
        # Если фоны не найдены, возвращаем черный фон
        print("Предупреждение: Не найдены фоновые изображения. Используется черный фон.")
        return Image.new('RGB', canvas_size, color='black').convert('RGBA')

    # Выбираем случайное фоновое изображение
    bg_path = random.choice(background_paths)
    bg_img = Image.open(bg_path).convert('RGB')
    
    canvas_width, canvas_height = canvas_size
    img_width, img_height = bg_img.size
    
    if img_width < canvas_width or img_height < canvas_height:
        # Если изображение меньше, просто растягиваем его до размера холста
        # Используем LANCZOS для лучшего качества при увеличении
        bg_img = bg_img.resize(canvas_size, Image.Resampling.LANCZOS)
        
    elif img_width > canvas_width and img_height > canvas_height:
        # Если изображение больше, вырезаем случайный фрагмент
        max_x = img_width - canvas_width
        max_y = img_height - canvas_height
        
        # Выбираем случайную верхнюю левую точку для обрезки
        crop_x = random.randint(0, max_x)
        crop_y = random.randint(0, max_y)
        
        bbox = (crop_x, crop_y, crop_x + canvas_width, crop_y + canvas_height)
        bg_img = bg_img.crop(bbox)
        
    else:
        # Если одна сторона совпадает, а другая нет (например, 1024x1500)
        # Просто изменяем размер, чтобы точно соответствовать холсту, это сохранит
        # большую часть контента, но может немного исказить пропорции.
        bg_img = bg_img.resize(canvas_size, Image.Resampling.LANCZOS)

    # Конвертируем в RGBA для возможности вставки прозрачных PNG
    return bg_img.convert('RGBA')

# --- Вспомогательные функции для инструментов (из предыдущего ответа) ---

def apply_random_transform(image):
    """Применяет случайные трансформации (поворот, зеркальное отражение, масштабирование) к изображению."""
    angle = random.choice([0, 90, 180, 270])
    if angle != 0:
        image = image.rotate(angle, expand=True)
    if random.choice([True, False]):
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
    if random.choice([False, False, True]): 
        image = image.transpose(Image.FLIP_TOP_BOTTOM)
    
    scale_factor = random.uniform(0.8, 1.2)
    new_width = int(image.width * scale_factor)
    new_height = int(image.height * scale_factor)
    image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)

    return image

def place_image_randomly(canvas, tool_image):
    """Случайным образом размещает инструмент на холсте."""
    max_x = canvas.width - tool_image.width
    max_y = canvas.height - tool_image.height
    
    if max_x < 0 or max_y < 0:
        return canvas, (0, 0, 0, 0) 
        
    x = random.randint(0, max_x)
    y = random.randint(0, max_y)
    
    position = (x, y)
    
    # Вставка PNG с использованием его альфа-канала
    canvas.paste(tool_image, position, tool_image)
    
    bbox = (x, y, x + tool_image.width, y + tool_image.height)
    return canvas, bbox

def get_all_other_tool_paths(masked_root_dir, target_folder_name):
    """Собирает пути ко всем PNG-инструментам, исключая те, что в целевой папке."""
    all_paths = []
    for tool_folder in Path(masked_root_dir).iterdir():
        if tool_folder.is_dir() and tool_folder.name != target_folder_name:
            all_paths.extend(list(tool_folder.glob("*.png")))
    return all_paths

# --- Основная функция генерации ---

def generate_synthetic_dataset(masked_root_dir="../dataset/masked_png_dataset", 
                               output_root_dir="../dataset/synthetic_dataset"):
    
    Path(output_root_dir).mkdir(exist_ok=True)
    
    # 1. Предварительный сбор всех возможных фоновых JPG
    all_background_paths = get_background_image_paths(RAW_DATASET_ROOT, TARGET_TOOL_FOLDERS)

    print("Начало генерации синтетического датасета с реалистичными фонами...")
    
    for target_folder_name in TARGET_TOOL_FOLDERS:
        target_path = Path(masked_root_dir) / target_folder_name
        
        if not target_path.is_dir():
            print(f"Папка {target_folder_name} не найдена. Пропускаем.")
            continue
            
        output_tool_path = Path(output_root_dir) / target_folder_name
        output_tool_path.mkdir(exist_ok=True)
        
        # Получаем пути ко всем инструментам, кроме тех, что в текущей целевой папке
        other_tool_paths = get_all_other_tool_paths(masked_root_dir, target_folder_name)
        
        print(f"\nОбработка целевой папки: {target_folder_name}")
        
        for image_path in target_path.glob("*.png"):
            try:
                # 2. Создаем холст: генерируем случайный фон 1024x1024
                canvas = generate_random_background(all_background_paths, CANVAS_SIZE)
                
                # Загружаем, трансформируем и размещаем исходный инструмент
                original_tool = Image.open(image_path).convert("RGBA")
                transformed_original_tool = apply_random_transform(original_tool)
                target_crop_size = transformed_original_tool.size 
                
                # Выбираем случайное количество других инструментов (от 0 до MAX_RANDOM_TOOLS)
                num_random_tools = random.randint(0, MAX_RANDOM_TOOLS)
                random_tool_paths = random.sample(other_tool_paths, min(num_random_tools, len(other_tool_paths)))
                
                # Бросаем случайные инструменты на холст
                for random_tool_path in random_tool_paths:
                    random_tool_img = Image.open(random_tool_path).convert("RGBA")
                    random_tool_img_transformed = apply_random_transform(random_tool_img)
                    canvas, _ = place_image_randomly(canvas, random_tool_img_transformed)
                    
                # Бросаем исходный инструмент на холст
                canvas, original_bbox = place_image_randomly(canvas, transformed_original_tool)
                
                if original_bbox[2] == original_bbox[0]:
                     print(f"   Предупреждение: Исходный инструмент {image_path.name} слишком большой для холста. Пропущено.")
                     continue
                
                # Обрезка по местоположению исходного инструмента (как и раньше)
                center_x = (original_bbox[0] + original_bbox[2]) // 2
                center_y = (original_bbox[1] + original_bbox[3]) // 2
                crop_width, crop_height = target_crop_size
                
                crop_left = max(0, center_x - crop_width // 2)
                crop_upper = max(0, center_y - crop_height // 2)
                crop_right = min(CANVAS_SIZE[0], crop_left + crop_width)
                crop_lower = min(CANVAS_SIZE[1], crop_upper + crop_height)

                if crop_right - crop_left < crop_width:
                    crop_left = max(0, crop_right - crop_width)
                if crop_lower - crop_upper < crop_height:
                    crop_upper = max(0, crop_lower - crop_height)
                
                final_bbox = (crop_left, crop_upper, crop_right, crop_lower)
                
                cropped_scene = canvas.crop(final_bbox).convert('RGB')
                
                # Сохранение
                output_filename = f"{image_path.stem}_scene_with_bg.jpg"
                output_path = output_tool_path / output_filename
                cropped_scene.save(output_path, "JPEG", quality=95)
                
                print(f"   Успешно создана сцена: {output_filename}. Размер: {cropped_scene.size}")

            except Exception as e:
                print(f"   Произошла ошибка при обработке {image_path.name}: {e}")

    print("\nГенерация синтетического датасета с реалистичными фонами завершена! 🎉")

# Запуск функции
if __name__ == "__main__":
    # Укажите корневую папку с вырезанными PNG и папку для сохранения результата
    generate_synthetic_dataset(
        masked_root_dir="../dataset/masked_png_dataset", 
        output_root_dir="../dataset/synthetic_dataset_with_bg" # Новая папка для результатов
    )

In [1]:
from torchvision.datasets import ImageFolder


DATA_PATH = "../dataset/dataset" 
full_image_folder = ImageFolder(root=DATA_PATH)
paths_and_labels = full_image_folder.samples # Список кортежей (путь, метка)
class_names = full_image_folder.classes


In [2]:
class_names

['1 Отвертка «-»',
 '10 Ключ рожковыйнакидной  ¾',
 '11 Бокорезы',
 '2 Отвертка «+»',
 '3 Отвертка на смещенный крест',
 '4 Коловорот',
 '5 Пассатижи контровочные',
 '6 Пассатижи',
 '7 Шэрница',
 '8 Разводной ключ',
 '9 Открывашка для банок с маслом',
 'Групповые для тренировки',
 'Инструменты с линейкой']

In [6]:
import os
import random
from pathlib import Path
from PIL import Image

# 1. Параметры
CANVAS_SIZE = (1024, 1024)
TARGET_TOOL_FOLDERS = ['1 Отвертка «-»',
 '10 Ключ рожковыйнакидной  ¾',
 '11 Бокорезы',
 '2 Отвертка «+»',
 '3 Отвертка на смещенный крест',
 '4 Коловорот',
 '5 Пассатижи контровочные',
 '6 Пассатижи',
 '7 Шэрница',
 '8 Разводной ключ',
 '9 Открывашка для банок с маслом',
]
MAX_RANDOM_TOOLS = 4
# НОВОЕ: Диапазон количества примеров для каждого исходного изображения
NUM_EXAMPLES_PER_IMAGE_RANGE = (3, 5) 
# Корень исходного датасета для фоновых изображений
RAW_DATASET_ROOT = "../dataset/dataset" 

# --- Вспомогательные функции для фонов (без изменений) ---

def get_background_image_paths(raw_root_dir, target_folders):
    """Собирает пути ко всем JPG-изображениям, исключая те, что в целевых папках."""
    all_background_paths = []
    
    for tool_folder in Path(raw_root_dir).iterdir():
        if tool_folder.is_dir() and tool_folder.name not in target_folders:
            # Собираем все JPG-файлы в этой папке
            all_background_paths.extend(list(tool_folder.glob("*.JPG")))
            
    return all_background_paths

def generate_random_background(background_paths, canvas_size):
    """
    Выбирает случайное фоновое изображение и обрабатывает его до размера холста (1024x1024).
    """
    if not background_paths:
        print("Предупреждение: Не найдены фоновые изображения. Используется черный фон.")
        return Image.new('RGB', canvas_size, color='black').convert('RGBA')

    bg_path = random.choice(background_paths)
    bg_img = Image.open(bg_path).convert('RGB')
    
    canvas_width, canvas_height = canvas_size
    img_width, img_height = bg_img.size
    
    if img_width < canvas_width or img_height < canvas_height:
        bg_img = bg_img.resize(canvas_size, Image.Resampling.LANCZOS)
    elif img_width > canvas_width and img_height > canvas_height:
        max_x = img_width - canvas_width
        max_y = img_height - canvas_height
        crop_x = random.randint(0, max_x)
        crop_y = random.randint(0, max_y)
        bbox = (crop_x, crop_y, crop_x + canvas_width, crop_y + canvas_height)
        bg_img = bg_img.crop(bbox)
    else:
        bg_img = bg_img.resize(canvas_size, Image.Resampling.LANCZOS)

    return bg_img.convert('RGBA')

# --- Вспомогательные функции для инструментов (без изменений) ---

def apply_random_transform(image):
    """Применяет случайные трансформации (поворот, зеркальное отражение, масштабирование) к изображению."""
    angle = random.choice([0, 90, 180, 270])
    if angle != 0:
        image = image.rotate(angle, expand=True)
    if random.choice([True, False]):
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
    if random.choice([False, False, True]): 
        image = image.transpose(Image.FLIP_TOP_BOTTOM)
    
    scale_factor = random.uniform(0.8, 1.2)
    new_width = int(image.width * scale_factor)
    new_height = int(image.height * scale_factor)
    image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)

    return image

def place_image_randomly(canvas, tool_image):
    """Случайным образом размещает инструмент на холсте."""
    max_x = canvas.width - tool_image.width
    max_y = canvas.height - tool_image.height
    
    if max_x < 0 or max_y < 0:
        return canvas, (0, 0, 0, 0) 
        
    x = random.randint(0, max_x)
    y = random.randint(0, max_y)
    
    position = (x, y)
    
    canvas.paste(tool_image, position, tool_image)
    
    bbox = (x, y, x + tool_image.width, y + tool_image.height)
    return canvas, bbox

def get_all_other_tool_paths(masked_root_dir, target_folder_name):
    """Собирает пути ко всем PNG-инструментам, исключая те, что в целевой папке."""
    all_paths = []
    for tool_folder in Path(masked_root_dir).iterdir():
        if tool_folder.is_dir() and tool_folder.name != target_folder_name:
            all_paths.extend(list(tool_folder.glob("*.png")))
    return all_paths

# --- Основная функция генерации (с изменениями) ---

def generate_synthetic_dataset(masked_root_dir="../dataset/masked_png_dataset", 
                                 output_root_dir="../dataset/synthetic_dataset"):
    
    Path(output_root_dir).mkdir(exist_ok=True)
    
    all_background_paths = get_background_image_paths(RAW_DATASET_ROOT, TARGET_TOOL_FOLDERS)

    print("Начало генерации синтетического датасета с реалистичными фонами...")
    
    for target_folder_name in TARGET_TOOL_FOLDERS:
        target_path = Path(masked_root_dir) / target_folder_name
        
        if not target_path.is_dir():
            print(f"Папка {target_folder_name} не найдена. Пропускаем.")
            continue
            
        output_tool_path = Path(output_root_dir) / target_folder_name
        output_tool_path.mkdir(exist_ok=True)
        
        other_tool_paths = get_all_other_tool_paths(masked_root_dir, target_folder_name)
        
        print(f"\nОбработка целевой папки: {target_folder_name}")
        
        for image_path in target_path.glob("*.png"):
            # ИЗМЕНЕНИЕ 1: Генерируем от 3 до 5 примеров для каждого исходного файла
            num_examples_to_generate = random.randint(*NUM_EXAMPLES_PER_IMAGE_RANGE)
            
            for i in range(num_examples_to_generate):
                try:
                    # Создаем холст
                    canvas = generate_random_background(all_background_paths, CANVAS_SIZE)
                    
                    # --- ИЗМЕНЕНИЕ 2: Логика размещения со случайным порядком слоев ---
                    
                    tools_to_place = []
                    
                    # 1. Подготавливаем целевой инструмент и добавляем его в общий список
                    original_tool = Image.open(image_path).convert("RGBA")
                    transformed_original_tool = apply_random_transform(original_tool)
                    tools_to_place.append({'image': transformed_original_tool, 'is_target': True})
                    
                    # 2. Подготавливаем случайные инструменты и добавляем их в тот же список
                    num_random_tools = random.randint(0, MAX_RANDOM_TOOLS)
                    random_tool_paths = random.sample(other_tool_paths, min(num_random_tools, len(other_tool_paths)))
                    
                    for random_tool_path in random_tool_paths:
                        random_tool_img = Image.open(random_tool_path).convert("RGBA")
                        random_tool_img_transformed = apply_random_transform(random_tool_img)
                        tools_to_place.append({'image': random_tool_img_transformed, 'is_target': False})
                        
                    # 3. Перемешиваем список, чтобы порядок наложения был случайным
                    random.shuffle(tools_to_place)
                    
                    # 4. Размещаем все инструменты на холсте в случайном порядке
                    original_bbox = None
                    for tool_data in tools_to_place:
                        canvas, bbox = place_image_randomly(canvas, tool_data['image'])
                        # Если это был целевой инструмент, сохраняем его bbox для последующей обрезки
                        if tool_data['is_target']:
                            original_bbox = bbox
                            target_crop_size = tool_data['image'].size # Размер нужен для обрезки
                    
                    # --- Конец изменений в логике размещения ---

                    if not original_bbox or original_bbox[2] == original_bbox[0]:
                        print(f"   [Пример {i+1}/{num_examples_to_generate}] Предупреждение: Целевой инструмент {image_path.name} слишком большой. Пропущено.")
                        continue
                    
                    # Обрезка по местоположению исходного инструмента (логика осталась прежней)
                    center_x = (original_bbox[0] + original_bbox[2]) // 2
                    center_y = (original_bbox[1] + original_bbox[3]) // 2
                    crop_width, crop_height = target_crop_size
                    
                    crop_left = max(0, center_x - crop_width // 2)
                    crop_upper = max(0, center_y - crop_height // 2)
                    crop_right = min(CANVAS_SIZE[0], crop_left + crop_width)
                    crop_lower = min(CANVAS_SIZE[1], crop_upper + crop_height)

                    if crop_right - crop_left < crop_width:
                        crop_left = max(0, crop_right - crop_width)
                    if crop_lower - crop_upper < crop_height:
                        crop_upper = max(0, crop_lower - crop_height)
                    
                    final_bbox = (crop_left, crop_upper, crop_right, crop_lower)
                    cropped_scene = canvas.crop(final_bbox).convert('RGB')
                    
                    # ИЗМЕНЕНИЕ 3: Новое имя файла, чтобы избежать перезаписи
                    output_filename = f"{image_path.stem}_scene_{i+1}_with_bg.jpg"
                    output_path = output_tool_path / output_filename
                    cropped_scene.save(output_path, "JPEG", quality=95)
                    
                    print(f"   [Пример {i+1}/{num_examples_to_generate}] Успешно создана сцена: {output_filename}. Размер: {cropped_scene.size}")

                except Exception as e:
                    print(f"   [Пример {i+1}/{num_examples_to_generate}] Произошла ошибка при обработке {image_path.name}: {e}")

    print("\nГенерация синтетического датасета с реалистичными фонами завершена! 🎉")

# Запуск функции
if __name__ == "__main__":
    generate_synthetic_dataset(
        masked_root_dir="../dataset/masked_png_dataset", 
        output_root_dir="../dataset/synthetic_dataset_with_bg_alltools"
    )

Начало генерации синтетического датасета с реалистичными фонами...

Обработка целевой папки: 1 Отвертка «-»
   [Пример 1/3] Успешно создана сцена: DSCN4831_masked_scene_1_with_bg.jpg. Размер: (462, 255)
   [Пример 2/3] Успешно создана сцена: DSCN4831_masked_scene_2_with_bg.jpg. Размер: (434, 240)
   [Пример 3/3] Успешно создана сцена: DSCN4831_masked_scene_3_with_bg.jpg. Размер: (324, 586)
   [Пример 1/3] Успешно создана сцена: DSCN2472_masked_scene_1_with_bg.jpg. Размер: (513, 284)
   [Пример 2/3] Успешно создана сцена: DSCN2472_masked_scene_2_with_bg.jpg. Размер: (330, 596)
   [Пример 3/3] Успешно создана сцена: DSCN2472_masked_scene_3_with_bg.jpg. Размер: (278, 502)
   [Пример 1/4] Успешно создана сцена: DSCN2512_masked_scene_1_with_bg.jpg. Размер: (438, 237)
   [Пример 2/4] Успешно создана сцена: DSCN2512_masked_scene_2_with_bg.jpg. Размер: (274, 507)
   [Пример 3/4] Успешно создана сцена: DSCN2512_masked_scene_3_with_bg.jpg. Размер: (306, 565)
   [Пример 4/4] Успешно создана сцена